In [ ]:
# graph.py

class Graph:

    # Constructor
    def __init__(self):
        self.graph = {}
        self.nodeCoords = {}

    # Adds an edge to the graph.
    def addEdge(self, node1, node2, w=1):
        # If node1 and/or node2 not in graph, add them.
        if node1 not in self.graph:
            self.graph[node1] = []
        if node2 not in self.graph:
            self.graph[node2] = []
        # Add edge between node1 and node2.
        self.graph[node1].append((node2, w))
        self.graph[node2].append((node1, w))

    # Adds a node to the graph.
    def addNode(self, node, x, y):
        self.nodeCoords[node] = (x, y)

In [ ]:
# utils.py

# Importing Libraries
import csv

# Read .txt file and return list of edges.
def readEdgeList(filename):
    edges = []
    with open(filename, 'r') as file:
        for line in file:
            n1, n2, w = line.strip().split(',')
            edges.append((n1, n2, float(w)))
    return edges

# Read .csv file and return node coordinates.
def readNodeCoords(filename):
    nodes = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            n1, x, y = row
            nodes.append((n1, float(x), float(y)))
    return nodes

In [ ]:
# searchAlgorithms.py

# Importing Libraries
from collections import deque
import heapq
import math

# BFS Algorithm
def bfs(graph, start, goal):
    # Start queue with start node.
    queue = deque([start])
    # List that holds visited nodes.
    visited = []

    # While queue is not empty...
    while queue:
        # Dequeue node
        node = queue.popleft()
        # Skip if node was already visited.
        if node in visited:
            continue
        # Add node to visited list.
        visited.append(node)

        # If the goal node is reached, return the visited nodes.
        if node == goal:
            return visited

        # Gets neighbors of node
        for neighbor, cost in graph.graph.get(node, []):
            # Add neighbor to queue if not visited.
            if neighbor not in visited:
                queue.append(neighbor)
    # Return visited nodes if goal not found.
    return visited

# DFS Algorithm
def dfs(graph, start, goal):
    # Start stack with start node.
    stack = [start]
    # List that holds visited nodes.
    visited = []

    # While stack is not empty...
    while stack:
        # Pop node from stack
        node = stack.pop()
        # Skip if node was already visited.
        if node in visited:
            continue
        # Add node to visited list.
        visited.append(node)

        # If the goal node is reached, return the visited nodes.
        if node == goal:
            return visited

        # Gets neighbors of node
        for neighbor, cost in graph.graph.get(node, []):
            # Add neighbor to stack if not visited.
            if neighbor not in visited:
                stack.append(neighbor)
    # Return visited nodes if goal not found.
    return visited

# A* Search Algorithm and Heuristics

# Euclidean Distance Heuristic
def eucDistance(node, goal, nodePositions):
    x1, y1 = nodePositions[node]
    x2, y2 = nodePositions[goal]
    distance = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

    # ---------- COMMENT THIS WHEN DONE TESTING ----------
    # print(f"Euclidean Distance from {node} to {goal}: {distance}")

    return distance

# Manhattan Distance Heuristic
def manDistance(node, goal, nodePositions):
    x1, y1 = nodePositions[node]
    x2, y2 = nodePositions[goal]
    distance = abs(x1 - x2) + abs(y1 - y2)

    # ---------- COMMENT THIS WHEN DONE TESTING ----------
    # print(f"Manhattan Distance from {node} to {goal}: {distance}")

    return distance

# Chebyshev Distance Heuristic
def chebDistance(node, goal, nodePositions):
    x1, y1 = nodePositions[node]
    x2, y2 = nodePositions[goal]
    distance = max(abs(x1 - x2), abs(y1 - y2))

    # ---------- COMMENT THIS WHEN DONE TESTING ----------
    # print(f"Chebyshev Distance from {node} to {goal}: {distance}")

    return distance

# A* Search
def aStar(graph, start, goal, nodePositions, heuristic):
    # Priority queue for starting node.
    frontier = []
    heapq.heappush(frontier, (0, start))

    # Cost to reach each node
    costSoFar = {start: 0}

    # Path taken to reach each node
    cameFrom = {start: None}

    # While frontier is not empty...
    while frontier:
        # Get node with lowest cost + heuristic
        currentCost, currentNode = heapq.heappop(frontier)

        # ---------- COMMENT THIS WHEN DONE TESTING ----------
        # print(f"Current Node: {currentNode}, Cost: {currentCost}")

        # If goal is reached, return path
        if currentNode == goal:
            path = []
            while currentNode:
                path.append(currentNode)
                currentNode = cameFrom[currentNode]
            path.reverse()
            return path

        # Go through neighbors
        for neighbor, cost in graph.graph.get(currentNode, []):

            # Calculate new cost
            newCost = costSoFar[currentNode] + cost

            if neighbor not in costSoFar or newCost < costSoFar[neighbor]:
                costSoFar[neighbor] = newCost
                priority = newCost + heuristic(neighbor, goal, nodePositions)
                heapq.heappush(frontier, (priority, neighbor))
                cameFrom[neighbor] = currentNode

                # ---------- COMMENT THIS WHEN DONE TESTING ----------
                # print(f"Next Node: {neighbor}, New Cost: {newCost}, Priority: {priority}")

    return list(cameFrom.keys())

In [52]:
# main.py
from google.colab import files

# Choose proper files to upload
# This is main.py, searchAlgorithms.py, utils.py, graph.py
# and the six TestCase_XX files.
files.upload()


# Import Libraries
import time
from graph import Graph
from searchAlgorithms import bfs, dfs, aStar, eucDistance, manDistance, chebDistance
from utils import readEdgeList, readNodeCoords

# Main Function
def main():
    # Prompt user for test case names
    testCase = input("Enter test case name (e.g. TestCase_01): ")
    print()

    # Gets the file from user input
    edgeListFile = f"{testCase}_EdgeList.txt"
    nodeCoordsFile = f"{testCase}_NodeID.csv"

    # Read edge list and node coords.
    edges = readEdgeList(edgeListFile)
    nodes = readNodeCoords(nodeCoordsFile)

    # Create graph, adding edges and nodes.
    graph = Graph()
    for node, x, y in nodes:
        graph.addNode(node, x, y)
    for n1, n2, w in edges:
        graph.addEdge(n1, n2, w)

    # Run the algorithms.
    startNode = nodes[0][0] # First node in NodeID file.
    goalNode = nodes[-1][0] # Last node in NodeID file.

    # Get node positions.
    nodePositions = {node: (x, y) for node, x, y in nodes}

    # Run and Time BFS algorithm.
    startTime = time.time()
    bfsVisitedNodes = bfs(graph, startNode, goalNode)
    bfsTime = time.time() - startTime
    # Run and Time DFS algorithm.
    startTime = time.time()
    dfsVisitedNodes = dfs(graph, startNode, goalNode)
    dfsTime = time.time() - startTime
    # Run and Time A* algorithm with Euclidean distance heuristic.
    startTime = time.time()
    aStarVisitedNodesEuc = aStar(graph, startNode, goalNode, nodePositions, eucDistance)
    aStarEucTime = time.time() - startTime
    # Run and Time A* algorithm with Manhattan distance heuristic.
    startTime = time.time()
    aStarVisitedNodesMan = aStar(graph, startNode, goalNode, nodePositions, manDistance)
    aStarManTime = time.time() - startTime
    # Run and Time A* algorithm with Chebyshev distance heuristic.
    startTime = time.time()
    aStarVisitedNodesCheb = aStar(graph, startNode, goalNode, nodePositions, chebDistance)
    aStarChebTime = time.time() - startTime

    # Print the list of states visited by the algorithms.
    print("BFS:", bfsVisitedNodes)
    print("DFS:", dfsVisitedNodes)
    print("A* with Euclidean Distance:", aStarVisitedNodesEuc)
    print("A* with Manhattan Distance:", aStarVisitedNodesMan)
    print("A* with Chebyshev Distance:", aStarVisitedNodesCheb)

    # Print the time taken by each algorithm
    print("\nTime taken by each algorithm:")
    print(f"BFS: {bfsTime:.6f} seconds")
    print(f"DFS: {dfsTime:.6f} seconds")
    print(f"A* with Euclidean Distance: {aStarEucTime:.6f} seconds")
    print(f"A* with Manhattan Distance: {aStarManTime:.6f} seconds")
    print(f"A* with Chebyshev Distance: {aStarChebTime:.6f} seconds")

if __name__ == "__main__":
    main()

Saving graph.py to graph.py
Saving main.py to main.py
Saving searchAlgorithms.py to searchAlgorithms.py
Saving TestCase_01_EdgeList.txt to TestCase_01_EdgeList.txt
Saving TestCase_01_NodeID.csv to TestCase_01_NodeID.csv
Saving TestCase_02_EdgeList.txt to TestCase_02_EdgeList.txt
Saving TestCase_02_NodeID.csv to TestCase_02_NodeID.csv
Saving TestCase_03_EdgeList.txt to TestCase_03_EdgeList.txt
Saving TestCase_03_NodeID.csv to TestCase_03_NodeID.csv
Saving utils.py to utils.py
Enter test case name (e.g. TestCase_01): TestCase_01

BFS: ['N_0', 'N_1', 'N_6', 'N_2', 'N_5', 'N_7', 'N_3', 'N_10', 'N_12', 'N_11', 'N_15', 'N_13', 'N_17', 'N_16', 'N_20', 'N_14', 'N_8', 'N_18', 'N_22', 'N_21', 'N_9', 'N_19', 'N_23', 'N_4', 'N_24']
DFS: ['N_0', 'N_1', 'N_2', 'N_3', 'N_6', 'N_7', 'N_12', 'N_17', 'N_22', 'N_23', 'N_13', 'N_18', 'N_19', 'N_24']
A* with Euclidean Distance: ['N_0', 'N_1', 'N_6', 'N_7', 'N_12', 'N_13', 'N_18', 'N_19', 'N_24']
A* with Manhattan Distance: ['N_0', 'N_1', 'N_6', 'N_7', 'N_1